In [19]:
# fill of 10 day gap end of oct. 2017 with  data average from other years

# load data
import pandas as pd
%pip install scipy

weather = pd.read_csv("C:/Users/julia/Documents/Uni_lokal/ML/wetter.csv", sep=',')
weather['Datum'] = pd.to_datetime(weather['Datum'])
# set index to date for time interpolation
weather = weather.set_index('Datum')
# interpolate using time method, make filled values also intergers
weather['bewoelkung_time'] = weather['Bewoelkung'].interpolate(method='time').round().astype(int)


print(weather[['Bewoelkung', 'bewoelkung_time']].loc['2017-10-29':'2017-11-09'])    
print(weather.tail())

Note: you may need to restart the kernel to use updated packages.
            Bewoelkung  bewoelkung_time
Datum                                  
2017-10-29         2.0                2
2017-10-30         NaN                2
2017-10-31         NaN                3
2017-11-01         NaN                3
2017-11-02         NaN                3
2017-11-03         NaN                4
2017-11-04         NaN                4
2017-11-05         NaN                5
2017-11-06         NaN                5
2017-11-07         NaN                5
2017-11-08         NaN                6
2017-11-09         6.0                6
            Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
Datum                                                                 
2019-07-28         3.0     23.3500                   14         5.0   
2019-07-29         6.0     25.2500                    7        61.0   
2019-07-30         7.0     20.7375                    8        61.0   
2019-07-31         


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# fill the data gaps in wetter_code with KNN imputation with other features
#from sklearn.impute import KNNImputer
%pip install scikit-learn
%pip install KNNImputer
from sklearn.impute import KNNImputer

features = ['Wettercode', 'Temperatur', 'bewoelkung_time', 'Windgeschwindigkeit']
imputer = KNNImputer(n_neighbors=5)

imputed_data = imputer.fit_transform(weather[features])

weather['wettercode_knn'] = imputed_data[:, 0].round().astype(int)


print(weather[['Wettercode', 'wettercode_knn']].loc['2017-10-29':'2017-11-09'])
                                                            



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
            Wettercode  wettercode_knn
Datum                                 
2017-10-29        21.0              21
2017-10-30         NaN              22
2017-10-31         NaN               6
2017-11-01         NaN              38
2017-11-02         NaN               6
2017-11-03         NaN               6
2017-11-04         NaN               7
2017-11-05         NaN              27
2017-11-06         NaN              31
2017-11-07         NaN              36
2017-11-08         NaN              36
2017-11-09        10.0              10


ERROR: Could not find a version that satisfies the requirement KNNImputer (from versions: none)
ERROR: No matching distribution found for KNNImputer

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# fill values with random forest (rf) imputation 
#weather[['wettercode_irmi']] = iter_imputer.fit_transform(weather[['Wettercode']]).round().astype(int)
#print(weather[['Wettercode', 'wettercode_irmi']].loc['2017-10-29':'2017-11-09'])

# Iterative Robust Model Imputation (IRMI) but with random forest for categorial data

#alternative approach to fill missing wettercode data
#weathercode has categorical values --> use random forest
#irmi with random forest classifier not possible since temp etc. continous data --> use only wether code and no additional features

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier

# Imputer für kategoriale Daten
imputer_rf = IterativeImputer(
    estimator=RandomForestClassifier(n_estimators=10, random_state=0),
    random_state=0,
    max_iter=10
)


# Imputieren
imputed_data = imputer_rf.fit_transform(weather[['Wettercode']])

# Als neue Spalte hinzufügen
weather['Wettercode_rf'] = imputed_data[:, 0].round().astype(int)

# Vergleich anzeigen
print(weather[['Wettercode', 'Wettercode_rf']].loc['2017-10-29':'2017-11-09'])

            Wettercode  Wettercode_rf
Datum                                
2017-10-29        21.0             21
2017-10-30         NaN             37
2017-10-31         NaN             37
2017-11-01         NaN             37
2017-11-02         NaN             37
2017-11-03         NaN             37
2017-11-04         NaN             37
2017-11-05         NaN             37
2017-11-06         NaN             37
2017-11-07         NaN             37
2017-11-08         NaN             37
2017-11-09        10.0             10


In [22]:
# show only rows where values of wettercode_knn and wettercode_rf differ
diff_rows = weather[weather['wettercode_knn'] != weather['Wettercode_rf']]
print(diff_rows[['Wettercode', 'wettercode_knn', 'Wettercode_rf']])   

            Wettercode  wettercode_knn  Wettercode_rf
Datum                                                
2012-01-02         NaN              33             37
2012-01-06         NaN              18             37
2012-01-10         NaN              45             37
2012-01-13         NaN              14             37
2012-01-14         NaN              18             37
...                ...             ...            ...
2019-06-28         NaN               2             37
2019-07-07         NaN              68             37
2019-07-10         NaN               6             37
2019-07-15         NaN              64             37
2019-07-25         NaN              53             37

[653 rows x 3 columns]


In [23]:
#KNN Imputation better approach than RF Imputation in this case

# fill the data gaps in wetter_code with KNN imputation with other features 
#delete Wettercode_rf column from wather dataframe
weather = weather.drop(columns=['Wettercode_rf'])

print(weather.head())

            Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
Datum                                                                 
2012-01-01         8.0      9.8250                   14        58.0   
2012-01-02         7.0      7.4375                   12         NaN   
2012-01-03         8.0      5.5375                   18        63.0   
2012-01-04         4.0      5.6875                   19        80.0   
2012-01-05         6.0      5.3000                   23        80.0   

            bewoelkung_time  wettercode_knn  
Datum                                        
2012-01-01                8              58  
2012-01-02                7              33  
2012-01-03                8              63  
2012-01-04                4              80  
2012-01-05                6              80  


In [24]:
#print head of dataframe


#export dataframe to csv
weather.to_csv("C:/Users/julia/Documents/Uni_lokal/ML/exported/wetter_filled.csv", index=True, sep=';' )